### QOSF August 2021 Challenge Attempt Using Qisikit

This is my attempt at the QOSF August 2021 challenge using Qiskit. The circuit updated via the VQA is the linear construction of the W-state described in here https://arxiv.org/pdf/1807.05572.pdf (the reference linked within the challenge prompt) with the controlled U3 gates left as parameters for the VQA to solve.

In [ ]:
# !pip install qiskit
!pip show qiskit

In [ ]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute
from qiskit.circuit import Parameter
from scipy.optimize import minimize
import math

## Level 1: 4-Qubit W state with VQA


In [ ]:
def vqa_wcirc4():
  
  n = 4

  # loss function
  def compute_expectation(counts, shots):
    
    error = 0
    for bitstring, count in counts.items():
        
        error += (shots/n - count) ** 2
        
    return error

  # W-state circuit generated from theta (where theta are the parameters updated)
  def create_circ(theta):
    qc = QuantumCircuit(n)
   
    # initial_state
    qc.x(0)
    
    # Preparation of the w-state by "spreading" the |1>
    qc.cu3(theta[0], 0, 0, 0, 1)
    qc.cx(1, 0)
    qc.cu3(theta[1], 0, 0, 0, 2)
    qc.cx(2, 0)
    qc.cu3(theta[2], 0, 0, 1, 3)
    qc.cx(3, 1)
            
    qc.measure_all()
        
    return qc

  # step function
  def get_expectation(shots=1024):
    
    backend = Aer.get_backend('qasm_simulator')
    backend.shots = shots
    
    def execute_circ(theta):
        
        qc = create_circ(theta)
        counts = backend.run(qc, seed_simulator=10, 
                             nshots=shots).result().get_counts()
        
        return compute_expectation(counts, shots)
    
    return execute_circ

  # return the optimal parameters for the U3 gates
  def solve():
    expectation = get_expectation()

    res = minimize(expectation, [1.0 for _ in range(n - 1)], method='COBYLA')
    print(res)
    return res.x

  solution = create_circ(solve())
  print(solution)
  backend = Aer.get_backend('qasm_simulator')
  backend.shots = 1024
    
  counts = backend.run(solution, seed_simulator=10, nshots=1024).result().get_counts()

  print(counts)

vqa_wcirc4()

     fun: 18.0
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 48
  status: 1
 success: True
       x: array([1.57121605, 1.50845381, 1.57759015])
        ┌───┐                  ┌───┐                                    ┌───┐»
   q_0: ┤ X ├────────■─────────┤ X ├────────■───────────────────────────┤ X ├»
        └───┘┌───────┴────────┐└─┬─┘        │                           └─┬─┘»
   q_1: ─────┤ U3(1.5712,0,0) ├──■──────────┼─────────────────■───────────┼──»
             └────────────────┘     ┌───────┴────────┐        │           │  »
   q_2: ────────────────────────────┤ U3(1.5085,0,0) ├────────┼───────────■──»
                                    └────────────────┘┌───────┴────────┐     »
   q_3: ──────────────────────────────────────────────┤ U3(1.5776,0,0) ├─────»
                                                      └────────────────┘     »
meas: 4/═════════════════════════════════════════════════════════════════════»
                                     

## Level 2: N-Qubit W state with VQA

In [ ]:
def vqa_wcirc(n):

  # loss function
  def compute_expectation(counts, shots):
    
    error = 0
    for bitstring, count in counts.items():
        
        error += (shots/n - count) ** 2
        
    return error

  # W-state circuit generated from theta (where theta are the parameters updated)
  def create_circ(theta):
    qc = QuantumCircuit(n)
   
    # initial_state
    qc.x(0)
    
    # linear construction 
    for i in range(len(theta)):
      qc.cu3(theta[i], 0, 0, i, i+1)
      qc.cx(i+1, i)
            
    qc.measure_all()
        
    return qc

  def get_expectation(shots=1024):
    
    backend = Aer.get_backend('qasm_simulator')
    backend.shots = shots
    
    def execute_circ(theta):
        
        qc = create_circ(theta)
        counts = backend.run(qc, seed_simulator=10, 
                             nshots=shots).result().get_counts()
        
        return compute_expectation(counts, shots)
    
    return execute_circ

  # return the optimal parameters for the U3 gates
  def solve():
    expectation = get_expectation()

    res = minimize(expectation, [1.0 for _ in range(n - 1)], method='COBYLA')
    print(res)
    return res.x

  solution = create_circ(solve())
  print(solution)
  backend = Aer.get_backend('qasm_simulator')
  backend.shots = 1024
    
  counts = backend.run(solution, seed_simulator=10, nshots=1024).result().get_counts()

  print(counts)


vqa_wcirc(6)

     fun: 9.333333333333334
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 67
  status: 1
 success: True
       x: array([2.26305016, 2.21087629, 2.06855155, 1.9219506 , 1.56538604])
        ┌───┐                  ┌───┐                                         »
   q_0: ┤ X ├────────■─────────┤ X ├─────────────────────────────────────────»
        └───┘┌───────┴────────┐└─┬─┘                  ┌───┐                  »
   q_1: ─────┤ U3(2.2631,0,0) ├──■──────────■─────────┤ X ├──────────────────»
             └────────────────┘     ┌───────┴────────┐└─┬─┘                  »
   q_2: ────────────────────────────┤ U3(2.2109,0,0) ├──■──────────■─────────»
                                    └────────────────┘     ┌───────┴────────┐»
   q_3: ───────────────────────────────────────────────────┤ U3(2.0686,0,0) ├»
                                                           └────────────────┘»
   q_4: ─────────────────────────────────────────────────────────────────────»


## Bonus: Implementing the W-state construction found in the [reference](https://arxiv.org/pdf/1807.05572.pdf).

In [ ]:
# generates a circuit that prepares the W-state for n qubits.
def create_wcirc(n):
  circ = QuantumCircuit(nqubits)
  circ.x(0)

  # generate the tree  
